## Objetivo do notebook
#### O objetivo deste notebook é realizar a leitura de dados das origens do dataset avaliações de turmas  na camada bronze e gravar de forma versionada na camada silver


### Import Libs

In [1]:
!pip install minio

In [2]:
from pyspark.sql import SparkSession
from datetime import datetime
from minio import Minio
from pyspark.sql.functions import to_date
from io import BytesIO

import os
import pandas as pd
import glob
import pyarrow.parquet as pq

### Definição de variáveis

In [3]:
# Sessão Spark
spark = SparkSession.builder.getOrCreate()

# Parametros de input e output das origens
camadaLeitura = 'bronze'
camadaEscrita = 'silver'
pasta = 'avaliacoes'
temp_blobs = '/home/jovyan/notebooks/temporary_blobs/' # pasta temporária para armazenamento de objetos

# Conexão ao miniIO
minio_endpoint = 'minio:9000'
minio_access_key = 'minioaccesskey'
minio_secret_key = 'miniosecretkey'
minio_client = Minio(minio_endpoint, access_key=minio_access_key, secret_key=minio_secret_key, secure=False)

# Definição de variáveis para versionamento dos dados na camada bronze
timestamp = datetime.today().strftime('%Y%m%d')
minio_path = f'{pasta}/{pasta + timestamp}'

### Leitura da camada Bronze

In [4]:
dataframes = []

try:
    # Lista objetos no bucket
    objects = minio_client.list_objects(camadaLeitura, prefix=pasta, recursive=True)

    # Itera sobre os objetos na pasta
    for obj in objects:
        # Lê o conteúdo do objeto
        content = minio_client.get_object(camadaLeitura, obj.object_name).read()
        
        # Lê o conteúdo do objeto Parquet usando pyarrow e converte para DataFrame Pandas
        df_pandas = pq.read_table(BytesIO(content)).to_pandas()
        
        # Adiciona o DataFrame Pandas à lista de DataFrames
        dataframes.append(df_pandas)
        
        print(f"Objeto encontrado: {obj.object_name}")
except error.MinioException as e:
    print(f"Erro ao listar objetos: {e}")

# Concatena todos os DataFrames Pandas em um único DataFrame Pandas
consolidated_df_pandas = pd.concat(dataframes, ignore_index=True)

# Converte para DataFrame PySpark
df_avaliacoes = spark.createDataFrame(consolidated_df_pandas)

Objeto encontrado: avaliacoes / avaliacoes20240209/part-00000-dcc78927-b299-45df-a4d1-5b3cf45ce3d6-c000.snappy.parquet
Objeto encontrado: avaliacoes / avaliacoes20240209/part-00001-dcc78927-b299-45df-a4d1-5b3cf45ce3d6-c000.snappy.parquet
Objeto encontrado: avaliacoes / avaliacoes20240209/part-00002-dcc78927-b299-45df-a4d1-5b3cf45ce3d6-c000.snappy.parquet
Objeto encontrado: avaliacoes / avaliacoes20240209/part-00003-dcc78927-b299-45df-a4d1-5b3cf45ce3d6-c000.snappy.parquet
Objeto encontrado: avaliacoes / avaliacoes20240209/part-00004-dcc78927-b299-45df-a4d1-5b3cf45ce3d6-c000.snappy.parquet
Objeto encontrado: avaliacoes / avaliacoes20240209/part-00005-dcc78927-b299-45df-a4d1-5b3cf45ce3d6-c000.snappy.parquet
Objeto encontrado: avaliacoes / avaliacoes20240209/part-00006-dcc78927-b299-45df-a4d1-5b3cf45ce3d6-c000.snappy.parquet
Objeto encontrado: avaliacoes / avaliacoes20240209/part-00007-dcc78927-b299-45df-a4d1-5b3cf45ce3d6-c000.snappy.parquet
Objeto encontrado: avaliacoes / avaliacoes202402

### Tratamento dos dados

In [5]:
df_avaliacoes = df_avaliacoes.select(
    'id_turma',
    'id_disciplina',
    to_date('data_avaliacao').alias('data_avaliacao'),
    'titulo',
    'etl_date'
)

### Gravação do dataframe em um diretório temporário

In [6]:
df_avaliacoes.write.parquet(temp_blobs, mode="overwrite")

### Gravação na camada Silver

In [7]:
# Recria bucket caso não exista
if not minio_client.bucket_exists(camadaEscrita):
    minio_client.make_bucket(camadaEscrita)

# Lista todos os arquivos Parquet no diretório temporário de blobs
arquivos_parquet = glob.glob(os.path.join(temp_blobs, '*.parquet'))

# Itera sobre a lista de arquivos Parquet e envia cada um para o MinIO
for arquivo_parquet in arquivos_parquet:
    try:
        # Envia o arquivo para a camada bronze
        nome_arquivo = os.path.basename(arquivo_parquet)
        minio_client.fput_object(camadaEscrita, os.path.join(minio_path, nome_arquivo), arquivo_parquet)
        print(f"Arquivo '{nome_arquivo}' enviado com sucesso para o MinIO em '{os.path.join(minio_path, nome_arquivo)}'.")
    except S3Error as e:
        print(f"Erro ao enviar o arquivo para o MinIO: {e}")

Arquivo 'part-00000-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet' enviado com sucesso para o MinIO em 'avaliacoes/avaliacoes20240209/part-00000-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet'.
Arquivo 'part-00001-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet' enviado com sucesso para o MinIO em 'avaliacoes/avaliacoes20240209/part-00001-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet'.
Arquivo 'part-00002-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet' enviado com sucesso para o MinIO em 'avaliacoes/avaliacoes20240209/part-00002-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet'.
Arquivo 'part-00003-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet' enviado com sucesso para o MinIO em 'avaliacoes/avaliacoes20240209/part-00003-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet'.
Arquivo 'part-00004-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet' enviado com sucesso para o MinIO em 'avaliacoes/avaliacoes2024

### Remove dados do diretório temporário de blobs

In [8]:
# Liste todos os arquivos na pasta
arquivos_na_pasta = os.listdir(temp_blobs)

# Itere sobre os arquivos e os delete
for arquivo in arquivos_na_pasta:
    caminho_completo = os.path.join(temp_blobs, arquivo)
    try:
        if os.path.isfile(caminho_completo):
            os.remove(caminho_completo)
            print(f'{caminho_completo} deletado com sucesso.')
    except Exception as e:
        print(f'Erro ao deletar {caminho_completo}: {e}')

spark.stop()

/home/jovyan/notebooks/temporary_blobs/.part-00000-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00001-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00002-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00003-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet.crc deletado com sucesso.


/home/jovyan/notebooks/temporary_blobs/.part-00004-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00005-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00006-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00007-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00008-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00009-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00010-25862720-3b5c-421f-af95-c4da6f0813c2-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00011-25862720-3b